In [1]:
import torch
from torch import nn
from torch import optim
import itertools
from tqdm import tqdm
from weather_prediction_full import WeatherPipeline

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def tensorfy(x):
    return torch.tensor(x, dtype=torch.float32)

In [4]:
train, val, test = WeatherPipeline().prepare_datasets(
    test_size=0.15,
    val_size=0.15,
    random_state=0,
    load_processed=True
)

In [5]:
class WeatherModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size_temp, output_size_wind):
        super(WeatherModel, self).__init__()

        layers = []
        for in_size, out_size in itertools.pairwise([input_size] + list(hidden_sizes)):
            layers.append(nn.Linear(in_size, out_size))
            layers.append(nn.LeakyReLU())

        self.fc1 = nn.Sequential(*layers)
        self.fc2_temp = nn.Linear(hidden_sizes[-1], output_size_temp)  # Dla temperatury (regresja)
        self.fc2_wind = nn.Linear(hidden_sizes[-1], output_size_wind)  # Dla wiatru (klasyfikacja)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = tensorfy(x)
        out = self.fc1(x)
        temp = self.fc2_temp(out)
        wind = self.sigmoid(self.fc2_wind(out))
        return temp.squeeze(), wind.squeeze()

In [6]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):
        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True


In [7]:
# Definicja funkcji trenowania modelu
def train_model(model, train_data, val_data,
               criterion_temp, criterion_wind, optimizer,
               num_epochs=100):
    X_train, y_temp_train, y_wind_train = train_data["X"], tensorfy(train_data["y_temp"]), tensorfy(train_data["y_wind"])
    X_val, y_temp_val, y_wind_val = val_data["X"], tensorfy(val_data["y_temp"]), tensorfy(val_data["y_wind"])
    history = {'train_loss': [], 'val_loss': []}
    early_stopper = EarlyStopping(tolerance=num_epochs//10, min_delta=0.01)
    pbar = tqdm(range(num_epochs), desc='Training')
    for epoch in pbar:
        model.train()
        optimizer.zero_grad()
        outputs_temp, outputs_wind = model(X_train)

        loss_temp = criterion_temp(outputs_temp, y_temp_train)
        loss_wind = criterion_wind(outputs_wind, y_wind_train)

        loss = loss_temp + loss_wind
        loss.backward()
        optimizer.step()

        # Walidacja
        model.eval()
        with torch.no_grad():
            val_outputs_temp, val_outputs_wind = model(X_val)
            val_loss_temp = criterion_temp(val_outputs_temp, y_temp_val)
            val_loss_wind = criterion_wind(val_outputs_wind, y_wind_val)
            val_loss = val_loss_temp + val_loss_wind

        history['train_loss'].append(loss.item())
        history['val_loss'].append(val_loss.item())

        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

        early_stopper(loss, val_loss)
        if early_stopper.early_stop:
            break

    return history

In [ ]:
# Trenowanie modelu dla Vancouver
city = 'Vancouver'
model = WeatherModel(
    input_size=train[city]['X'].shape[1],
    hidden_sizes=(),
    output_size_temp=1,
    output_size_wind=1
).to(device)

# Definicja funkcji strat
criterion_temp = nn.MSELoss()
criterion_wind = nn.BCELoss()

# Definicja optymalizatora
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Trenowanie modelu
history = train_model(
    model,
    train[city],
    val[city],
    criterion_temp,
    criterion_wind,
    optimizer,
    num_epochs=5000
)

Training:   1%|          | 50/5000 [00:00<00:19, 253.08it/s]

Epoch [10/5000], Loss: 0.8822, Val Loss: 0.8469
Epoch [20/5000], Loss: 0.7618, Val Loss: 0.7390
Epoch [30/5000], Loss: 0.7118, Val Loss: 0.6924
Epoch [40/5000], Loss: 0.6982, Val Loss: 0.6859
Epoch [50/5000], Loss: 0.6904, Val Loss: 0.6802
Epoch [60/5000], Loss: 0.6857, Val Loss: 0.6791


Training:   2%|▏         | 105/5000 [00:00<00:18, 265.63it/s]

Epoch [70/5000], Loss: 0.6829, Val Loss: 0.6776
Epoch [80/5000], Loss: 0.6806, Val Loss: 0.6775
Epoch [90/5000], Loss: 0.6786, Val Loss: 0.6779
Epoch [100/5000], Loss: 0.6768, Val Loss: 0.6787
Epoch [110/5000], Loss: 0.6751, Val Loss: 0.6795
Epoch [120/5000], Loss: 0.6734, Val Loss: 0.6803


Training:   3%|▎         | 162/5000 [00:00<00:17, 272.00it/s]

...

Epoch [500/5000], Loss: 0.6243, Val Loss: 0.6956
Epoch [510/5000], Loss: 0.6233, Val Loss: 0.6947
Epoch [520/5000], Loss: 0.6222, Val Loss: 0.6939
Epoch [530/5000], Loss: 0.6210, Val Loss: 0.6939
Epoch [540/5000], Loss: 0.6200, Val Loss: 0.6935


Training:  12%|█▏        | 578/5000 [00:02<00:18, 234.78it/s]

Epoch [550/5000], Loss: 0.6188, Val Loss: 0.6948
Epoch [560/5000], Loss: 0.6177, Val Loss: 0.6953
Epoch [570/5000], Loss: 0.6164, Val Loss: 0.6965
Epoch [580/5000], Loss: 0.6150, Val Loss: 0.6984
Epoch [590/5000], Loss: 0.6139, Val Loss: 0.7008


Training:  13%|█▎        | 632/5000 [00:02<00:17, 245.95it/s]

Epoch [600/5000], Loss: 0.6129, Val Loss: 0.7036
Epoch [610/5000], Loss: 0.6121, Val Loss: 0.7033
Epoch [620/5000], Loss: 0.6114, Val Loss: 0.7047
Epoch [630/5000], Loss: 0.6107, Val Loss: 0.7051


In [9]:
# Uzyskanie zestawu testowego
X_test_tensor = test['X']
y_temp_test_tensor = test['y_temp']
y_wind_test_tensor = test['y_wind']

# Przełącz model w tryb ewaluacji
model.eval()
with torch.no_grad():
    outputs_temp_test, outputs_wind_test = model(X_test_tensor)

# Konwersja na NumPy
y_temp_test_np = y_temp_test_tensor.numpy()
outputs_temp_test_np = outputs_temp_test.squeeze().numpy()

y_wind_test_np = y_wind_test_tensor.numpy()
outputs_wind_test_np = outputs_wind_test.squeeze().numpy()

# Predykcje klasyfikacyjne (próg 0.5)
y_wind_pred_np = (outputs_wind_test_np >= 0.5).astype(int)

KeyError: 'X'